# 03 – Alert Engine & Stock Health Dashboard
**Author:** Mousumi Paul | Feb 2025

## 1. Setup

In [ ]:
import pandas as pd, sys
sys.path.append('..')
from src.inventory_engine import run_optimization
from src.alert_engine import generate_alerts, print_alert_console, alert_summary_stats
print('✅ Ready')

## 2. Generate Alerts

In [ ]:
params = pd.read_csv('../data/raw/inventory_params.csv')
inv_df = run_optimization(params)
alerts = generate_alerts(inv_df)
print_alert_console(alerts)

## 3. Alert Summary Stats

In [ ]:
stats = alert_summary_stats(inv_df)
for k, v in stats.items():
    print(f'  {k:<28}: {v}')

## 4. Export Alert Report

In [ ]:
alerts.to_csv('../data/processed/low_stock_alerts.csv', index=False)
print('✅ Alert report saved: data/processed/low_stock_alerts.csv')

## 5. Visualise Alert Dashboard

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Current Stock vs ROP
cats = inv_df['Category'].tolist()
x = np.arange(len(cats))
axes[0].bar(x-0.2, inv_df['Current_Stock'], 0.4, label='Current Stock', color='#2E75B6', alpha=0.9)
axes[0].bar(x+0.2, inv_df['Reorder_Point'], 0.4, label='Reorder Point', color='#C0392B', alpha=0.9)
axes[0].set_xticks(x); axes[0].set_xticklabels(cats, rotation=15, ha='right')
axes[0].set_title('Current Stock vs Reorder Point', fontweight='bold')
axes[0].legend(); axes[0].grid(axis='y', alpha=0.3)

# Right: Alert status pie
status_counts = inv_df['Alert_Status'].value_counts()
colors_pie = {'🔴 CRITICAL – Below Safety Stock':'#C0392B',
              '🟠 REORDER NOW':'#E67E22',
              '🟡 EXCESS STOCK':'#D4A017',
              '🟢 HEALTHY':'#1E8449'}
pie_colors = [colors_pie.get(s,'#95A5A6') for s in status_counts.index]
axes[1].pie(status_counts.values, labels=status_counts.index,
            colors=pie_colors, autopct='%1.0f%%', startangle=90)
axes[1].set_title('Inventory Alert Distribution', fontweight='bold')

plt.suptitle('Supply Chain Dashboard – Stock Health Overview', fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../outputs/charts/alert_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Dashboard chart saved')